In [0]:
%sql
create schema if not exists sandbox.t_weetv

In [0]:
%sql
CREATE OR REPLACE TABLE sandbox.t_weetv.po_activation_mart
USING DELTA
PARTITIONED BY (date_ym)
AS
WITH T101 AS (
    /* KMS 관련 테이블 */
    SELECT 
      date_ym,
      mac,
      try_cast(po_date AS DATE) AS po_date,
      po_country_code,
      po_type,
      po_board_maker,
      po_odm,
      po_brand,
      po_country,
      po_year,
      po_platform
    FROM kic_data_ods.kms.kms_wee_tv
),
T102 AS (
    /* ACTIVATION: MAC별 최초 */
    SELECT
        s.mac_addr,
        s.activ_country_code,
        s.activ_date,
        s.activ_platform_code,
        s.activ_product_code,
        s.activ_sales_model
    FROM (
        SELECT
            T102.mac_addr,
            T102.Cntry_CODE   AS activ_country_code,
            DATE(T102.crt_date) AS activ_date,
            T102.Platform_code AS activ_platform_code,
            T102.Product_CODE  AS activ_product_code,
            T102.Sales_Model   AS activ_sales_model,
            ROW_NUMBER() OVER (
                PARTITION BY T102.mac_addr
                ORDER BY T102.crt_date ASC, T102.last_chg_date ASC, T102.he_etl_dt ASC
            ) AS rn
        FROM kic_data_ods.tlamp.activation_date T102
        WHERE EXISTS (
            SELECT 1
            FROM T101
            WHERE T101.mac = T102.mac_addr
        )
    ) s
    WHERE s.rn = 1
),
T103 AS (
    /* 앱 브랜드: MAC별 최초 */
    SELECT
        s.mac_addr,
        s.brand_name
    FROM (
        SELECT
            T103.mac_addr,
            T103.brand_name,
            ROW_NUMBER() OVER (
                PARTITION BY T103.mac_addr
                ORDER BY T103.src_file_date ASC, T103.he_etl_dt ASC
            ) AS rn
        FROM kic_data_ods.tlamp.apps_device_brand T103
        WHERE EXISTS (
            SELECT 1
            FROM T101
            WHERE T101.mac = T103.mac_addr
        )
    ) s
    WHERE s.rn = 1
)
SELECT
    -- 파티션 키
    T101.date_ym,

    -- PO 측 주요 컬럼
    T101.mac,
    T101.po_date,
    T101.po_country_code,
    T101.po_type,
    T101.po_board_maker,
    T101.po_odm,
    T101.po_brand,
    T101.po_country,
    T101.po_year,
    T101.po_platform,

    -- ACTIVATION 측
    T102.activ_date,
    T102.activ_country_code,
    T102.activ_platform_code,
    T102.activ_product_code,
    T102.activ_sales_model,

    -- BRAND 파싱
    SPLIT_PART(T103.brand_name, '_', 1)   AS activ_board_maker,
    SPLIT_PART(T103.brand_name, '_', 2)   AS activ_odm,
    SPLIT_PART(T103.brand_name, '_', 3)   AS activ_brand,

    -- 비교 지표
    CASE
        WHEN T101.po_country_code IS NULL THEN 'po_country_code_is_null'
        WHEN T102.activ_country_code IS NULL THEN 'not_yet_activated'
        WHEN T101.po_country_code <> T102.activ_country_code THEN 'N'
        ELSE 'Y'
    END                                      AS is_same_country,
    DATE_DIFF(T102.activ_date, T101.po_date) AS day_diff,
    
    current_timestamp() as he_etl_dt
FROM T101
LEFT JOIN T102
  ON T102.mac_addr = T101.mac
LEFT JOIN T103
  ON T103.mac_addr = T101.mac
;

In [0]:
%sql
-- 1) 테이블 생성
CREATE OR REPLACE TABLE sandbox.t_weetv.country_code_name_map (
    country_cd STRING COMMENT '국가 코드',
    country STRING COMMENT '국가명(약칭)',
    region STRING COMMENT 'Region 구분',
    remark STRING COMMENT '국가 전체명'
);

-- 2) 데이터 적재
INSERT INTO sandbox.t_weetv.country_code_name_map VALUES
('AE','UAE','MEA','United Arab Emirates'),
('AG','Antigua','LATAM','Antigua and Barbuda'),
('AN','Antilles','LATAM','Netherlands Antilles'),
('AS','AmericanSamoa','Asia','American Samoa'),
('BA','Bosnia','EU_Other','Bosnia and Herzegovina'),
('BF','BurkinaFaso','MEA','Burkina Faso'),
('BG','Bulgaria','EU_Other','Bulgaria'),
('BH','Bahrain','MEA','Bahrain'),
('BI','Burundi','MEA','Burundi'),
('BJ','Benin','MEA','Benin'),
('BM','Bermuda','EU_Other','Bermuda'),
('BN','Brunei','Asia','Brunei Darussalam'),
('BO','Bolivia','EU_Other','Bolivia'),
('BQ','BONAIRE','LATAM','Bonaire'),
('BR','Brazil','LATAM','Brazil'),
('BS','Bahamas','LATAM','Bahamas'),
('BT','Bhutan','Asia','Bhutan'),
('BV','Bouvet','EU_Other','Bouvet Island'),
('BW','Botswana','MEA','Botswana'),
('BY','Belarus','CIS','Belarus'),
('BZ','Belize','LATAM','Belize'),
('CA','Canada','US','Canada'),
('CC','Cocos','Asia','Cocos (Keeling) Island'),
('CD','DRCongo','MEA','Democratic Republic of the Congo'),
('CF','CentralAfricanRep','MEA','Central African Republic'),
('CG','REPCongo','MEA','Republic of the Congo'),
('CH','Switzerland','EU_Other','Switzerland'),
('CI','IvoryCoast','MEA','Cote dKIvoire(Republic of Ivory Coast)'),
('CK','CookIslands','Asia','Cook Islands'),
('CL','Chile','LATAM','Chile'),
('CM','Cameroon','MEA','Cameroon'),
('CN','China','Asia','China'),
('CO','Colombia','LATAM','Colombia'),
('CR','CostaRica','LATAM','Costa Rica'),
('CU','Cuba','LATAM','Cuba'),
('CV','CapeVerde','MEA','Cape Verde'),
('CW','CURACAO','LATAM','Curacao'),
('CX','Christmas Island','Asia','Christmas Island'),
('CY','Cyprus','EU_Other','Cyprus'),
('CZ','Czech','EU_Other','Czech Republic'),
('DE','Germany','EU_Top5','Germany'),
('DJ','Djibouti','MEA','Djibouti'),
('DK','Denmark','EU_Other','Denmark'),
('DM','Dominica','LATAM','Dominica'),
('DO','DominicanRep','LATAM','Dominican Republic'),
('DZ','Algeria','MEA','Algeria'),
('EC','Ecuador','LATAM','Ecuador'),
('EE','Estonia','EU_Other','Estonia'),
('EG','Egypt','MEA','Egypt'),
('EH','WesternSahara','MEA','Western Sahara'),
('ER','Eritrea','MEA','Eritrea'),
('ES','Spain','EU_Top5','Spain'),
('ET','Ethiopia','MEA','Ethiopia'),
('EU','EU','EU_Other','European Union'),
('FI','Finland','EU_Other','Finland'),
('FJ','Fiji','Asia','Fiji'),
('FK','Malvinas','EU_Other','Falkland Islands (Malvinas)'),
('FM','Micronesia','Asia','Micronesia, Federated States of Micronesia'),
('FO','Faeroe','EU_Other','Faeroe Islands'),
('FR','France','EU_Top5','France'),
('GA','Gabon','MEA','Gabon'),
('GB','UK','EU_Top5','United Kingdom'),
('GD','Grenada','LATAM','Grenada'),
('GE','Georgia','CIS','Georgia'),
('GF','FrenchGuiana','EU_Other','French Guiana'),
('GG','Guernsey','EU_Other','Guernsey'),
('GH','Ghana','MEA','Ghana'),
('GI','Gibraltar','EU_Other','Gibraltar'),
('GL','Greenland','EU_Other','Greenland'),
('GM','Gambia','MEA','Gambia'),
('GN','Guinea','MEA','Guinea'),
('GP','Guadeloupe','EU_Other','Guadeloupe'),
('GQ','EquatorialGuinea','MEA','Equatorial Guinea'),
('GR','Greece','EU_Other','Greece'),
('GS','SouthGeorgia','CIS','South georgia island'),
('GT','Guatemala','LATAM','Guatemala'),
('GU','Guam','US','Guam'),
('GW','GuineaBissau','MEA','Guinea-Bissau'),
('GY','Guyana','MEA','Guyana'),
('HK','HongKong','Asia','Hong Kong'),
('HM','HeardIsland','Asia','Heard Island'),
('HN','Honduras','LATAM','Honduras'),
('HR','Croatia','EU_Other','Croatia'),
('HT','Haiti','LATAM','Haiti'),
('HU','Hungary','EU_Other','Hungary'),
('ID','Indonesia','Asia','Indonesia'),
('IE','Ireland','EU_Other','Ireland'),
('IL','Israel','MEA','Israel'),
('IM','IsleofMan','EU_Other','Isle of Man'),
('IN','India','Asia_India','India'),
('IO','BritishIndian','EU_Other','British Indian ocean territory'),
('IQ','Iraq','MEA','Iraq'),
('IR','Iran','MEA','Iran'),
('IS','Iceland','EU_Other','Iceland'),
('IT','Italy','EU_Top5','Italy'),
('JE','Jersey','US','Jersey'),
('JM','Jamaica','LATAM','Jamaica'),
('JO','Jordan','MEA','Jordan'),
('JP','Japan','Asia','Japan'),
('KE','Kenya','MEA','Kenya'),
('KG','Kyrgyzstan','CIS','Kyrgyzstan'),
('KH','Cambodia','Asia','Cambodia'),
('KI','Kiribati','Asia','Kiribati'),
('KM','Comoros','MEA','Comoros'),
('KN','StKitts','LATAM','Saint Kitts and Nevis'),
('KR','Korea','Asia','Korea'),
('KW','Kuwait','MEA','Kuwait'),
('KY','CaymanIslands','EU_Other','Cayman Islands'),
('KZ','Kazakhstan','CIS','Kazakhstan'),
('LA','Laos','Asia','Lao Peoples Democratic Republic'),
('LB','Lebanon','MEA','Lebanon'),
('LC','StLucia','LATAM','Saint Lucia'),
('LI','Liechtenstein','EU_Other','Liechtenstein'),
('LK','SriLanka','Asia','Sri Lanka'),
('LR','Liberia','MEA','Liberia'),
('LS','Lesotho','MEA','Lesotho'),
('LT','Lithuania','EU_Other','Lithuania'),
('LU','Luxembourg','EU_Other','Luxembourg'),
('LV','Latvia','EU_Other','Latvia'),
('LY','Libya','MEA','Libyan Arab Jamahiriya'),
('MA','Morocco','MEA','Morocco'),
('MC','Monaco','EU_Other','Monaco'),
('MD','Moldova','CIS','Republic of Moldova'),
('ME','Montenegro','EU_Other','Montenegro'),
('MF','Saintmartin','EU_Other','Saint martin'),
('MG','Madagascar','MEA','Madagascar'),
('MH','MarshallIslands','Asia','Marshall Islands'),
('MK','Macedonia','EU_Other','Republic of Macedonia'),
('ML','Mali','MEA','Mali'),
('MM','Myanmar','Asia','Myanmar'),
('MN','Mongolia','Asia','Mongolia'),
('MO','Macao','Asia','Macao Special Administrative Region of China'),
('MP','NorthernMariana','US','Northern Mariana Islands'),
('MQ','Martinique','EU_Other','Martinique'),
('MR','Mauritania','MEA','Mauritania'),
('MS','Montserrat','EU_Other','Montserrat'),
('MT','Malta','EU_Other','Malta'),
('MU','Mauritius','MEA','Mauritius'),
('MV','Maldives','Asia','Maldives'),
('MW','Malawi','MEA','Malawi'),
('MX','Mexico','LATAM','Mexico'),
('MY','Malaysia','Asia','Malaysia'),
('MZ','Mozambique','MEA','Mozambique'),
('NA','Namibia','MEA','Namibia'),
('NC','NewCaledonia','Asia','New Caledonia'),
('NE','Niger','MEA','Niger'),
('NF','NorfolkIsland','Asia','Norfolk Island'),
('NG','Nigeria','MEA','Nigeria'),
('NI','Nicaragua','LATAM','Nicaragua'),
('NL','Netherlands','EU_Other','Netherlands'),
('NO','Norway','EU_Other','Norway'),
('NP','Nepal','Asia','Nepal'),
('NR','Nauru','Asia','Nauru'),
('NU','Niue','Asia','Niue'),
('NZ','NewZealand','Asia','New Zealand'),
('OM','Oman','MEA','Oman'),
('PA','Panama','LATAM','Panama'),
('PE','Peru','LATAM','Peru'),
('PF','FrenchPolynesia','EU_Other','French Polynesia'),
('PG','PapuaNewGuinea','Asia','Papua New Guinea'),
('PH','Philippines','Asia','Philippines'),
('PK','Pakistan','Asia','Pakistan'),
('PL','Poland','EU_Other','Poland'),
('PM','StPierre','EU_Other','Saint Pierre and Miquelon'),
('PN','Pitcairn','EU_Other','Pitcairn'),
('PR','PuertoRico','US','Puerto Rico'),
('PS','Palestinian','MEA','Occupied Palestinian Territory'),
('PT','Portugal','EU_Other','Portugal'),
('PW','Palau','Asia','Palau'),
('PY','Paraguay','LATAM','Paraguay'),
('QA','Qatar','MEA','Qatar'),
('RE','Reunion','MEA','Reunion'),
('RO','Romania','EU_Other','Romania'),
('RS','Serbia','EU_Other','Serbia'),
('RU','Russia','CIS','Russian Federation'),
('RW','Rwanda','MEA','Rwanda'),
('SA','SaudiArabia','MEA','Saudi Arabia'),
('SB','SolomonIslands','Asia','Solomon Islands'),
('SC','Seychelles','MEA','Seychelles'),
('SD','Sudan','MEA','Sudan'),
('SE','Sweden','EU_Other','Sweden'),
('SG','Singapore','Asia','Singapore'),
('SH','StHelena','EU_Other','Saint Helena'),
('SI','Slovenia','EU_Other','Slovenia'),
('SJ','Svalbard','EU_Other','Svalbard and Jan Mayen Islands'),
('SK','Slovakia','EU_Other','Slovakia'),
('SL','SierraLeone','MEA','Sierra Leone'),
('SM','SanMarino','EU_Other','San Marino'),
('SN','Senegal','MEA','Senegal'),
('SO','Somalia','MEA','Somalia'),
('SR','Suriname','MEA_Turkiye','Suriname'),
('ST','SaoTome','MEA','Sao Tome and Principe'),
('SV','ElSalvador','LATAM','El Salvador'),
('SY','Syria','MEA','Syrian Arab Republic'),
('SZ','Swaziland','MEA','Swaziland'),
('TC','TurksandCaicosIslands','EU_Other','Turks and Caicos Islands'),
('TD','Chad','MEA','Chad'),
('TF','FrenchSouthern','EU_Other','French southern territories'),
('TG','Togo','MEA','Togo'),
('TH','Thailand','Asia','Thailand'),
('TJ','Tajikistan','CIS','Tajikistan'),
('TK','Tokelau','Asia','Tokelau'),
('TM','Turkmenistan','CIS','Turkmenistan'),
('TN','Tunisia','MEA','Tunisia'),
('TO','Tonga','Asia','Tonga'),
('TP','EastTimor','Asia','East Timor'),
('TR','Turkiye','MEA_Turkiye','Turkiye'),
('TT','TrinidadandTobago','LATAM','Trinidad and Tobago'),
('TV','Tuvalu','Asia','Tuvalu'),
('TW','Taiwan','Asia','Taiwan'),
('TZ','Tanzania','MEA','United Republic of Tanzania'),
('UA','Ukraine','CIS','Ukraine'),
('UG','Uganda','MEA','Uganda'),
('UM','MinorOutlying','US','United States Minor Outlying Islands'),
('US','USA','US','U.S.A'),
('UY','Uruguay','LATAM','Uruguay'),
('UZ','Uzbekistan','CIS','Uzbekistan'),
('VA','VaticanCity','EU_Other','Holy See (Vatican city state)'),
('VC','StVincent','LATAM','Saint Vincent and the Grenadines'),
('VE','Venezuela','LATAM','Venezuela'),
('VG','BritishVirginIslands','EU_Other','British Virgin Islands'),
('VI','UsVirginIslands','US','United States Virgin Islands'),
('VN','Vietnam','Asia','Vietnam'),
('VU','Vanuatu','Asia','Vanuatu'),
('WF','Wallis','EU_Other','Wallis and Futuna Islands'),
('WS','Samoa','Asia','Samoa'),
('XK','KOSOVO','EU_Other','Kosovo'),
('YE','Yemen','MEA','Yemen'),
('YT','Mayotte','MEA','Mayotte'),
('ZA','SouthAfrica','MEA','South Africa'),
('ZM','Zambia','MEA','Zambia'),
('ZW','Zimbabwe','MEA','Zimbabwe');


In [0]:
%sql
CREATE OR REPLACE VIEW sandbox.t_weetv.v_po_activation_detail AS
SELECT
    a.mac                                            AS MAC,
    a.po_type                                        AS PO_Type,

    -- PO_Platform 매핑
    CASE a.po_platform
      WHEN '00' THEN 'C20P'
      WHEN '01' THEN 'C20L'
      WHEN '10' THEN 'C22P'
      WHEN '11' THEN 'C22L'
      WHEN '12' THEN 'C22H'
      WHEN '13' THEN 'C22T'
      WHEN '21' THEN 'C23L'
      WHEN '30' THEN 'C23T'
      WHEN '31' THEN 'C23U'
      WHEN '33' THEN 'C23T'
      WHEN '41' THEN 'C23V'
      WHEN '51' THEN 'C23U'
      WHEN '60' THEN 'C25P'
      WHEN '65' THEN 'C25H'
      WHEN '66' THEN 'C25H'
      WHEN '70' THEN 'C24T'
      ELSE a.po_platform
    END                                             AS PO_Platform,

    -- 국가코드 → 국명 (없으면 원래 코드 유지)
    COALESCE(b.country, a.po_country_code)          AS PO_Country,

    -- PO_Board_Maker 매핑
    CASE a.po_board_maker
      WHEN 'TT' THEN 'TOPTECH'
      WHEN 'MS' THEN 'MORESTAR'
      WHEN 'CV' THEN 'CULTRAVIEW'
      WHEN 'SP' THEN 'SILICONPLAYER'
      WHEN 'HK' THEN 'HIKEEN'
      ELSE a.po_board_maker
    END                                             AS PO_Board_Maker,

    a.po_odm                                        AS PO_ODM,
    a.po_brand                                      AS PO_Brand,
    a.po_date                                       AS PO_Date,

    -- Activation 측
    a.activ_platform_code                           AS Act_Platform,
    COALESCE(c.country, a.activ_country_code)       AS Act_Country,

    -- Act_Board_Maker 매핑
    CASE a.activ_board_maker
      WHEN 'TT' THEN 'TOPTECH'
      WHEN 'MS' THEN 'MORESTAR'
      WHEN 'CV' THEN 'CULTRAVIEW'
      WHEN 'SP' THEN 'SILICONPLAYER'
      WHEN 'HK' THEN 'HIKEEN'
      ELSE a.activ_board_maker
    END                                             AS Act_Board_Maker,

    a.activ_odm                                     AS Act_ODM,
    a.activ_brand                                   AS Act_Brand,
    a.activ_date                                    AS Act_Date,
    a.activ_sales_model                             AS Act_salesModel,

    a.date_ym                                       AS PO_date_ym,
    a.he_etl_dt                                     AS he_etl_dt
FROM sandbox.t_weetv.po_activation_mart a
LEFT JOIN sandbox.t_weetv.country_code_name_map b
       ON a.po_country_code = b.country_cd
LEFT JOIN sandbox.t_weetv.country_code_name_map c
       ON a.activ_country_code = c.country_cd
;

In [0]:
%sql
select 
  max(PO_Date), max(Act_Date), max(he_etl_dt)
from sandbox.t_weetv.v_po_activation_detail
where 1=1
  -- and Act_Date is null

In [0]:
%sql
select 
  count(1), count(distinct MAC)
from sandbox.t_weetv.v_po_activation_detail
where 1=1
  and PO_Date>='2023-01-01'
  and Act_Date>='2023-01-01'
  and (PO_Board_Maker != Act_Board_Maker 
    or PO_ODM != Act_ODM or PO_Brand != Act_Brand)
-- group by
--   PO_date_ym

In [0]:
%sql
create or replace view sandbox.z_yeswook_kim.rbt_mac
AS
(
select w60.*, 'webos60' as webos_version
from kic_data_private.tlamp.rbt_mac_addr_webos60 w60
union all
select w22.*, 'webos22' as webos_version
from kic_data_private.tlamp.rbt_mac_addr_webos22 w22
union all
select w23.*, 'webos23' as webos_version
from kic_data_private.tlamp.rbt_mac_addr_webos23 w23
union all
select w24.*, 'webos24' as webos_version
from kic_data_private.tlamp.rbt_mac_addr_webos24 w24
union all
select w25.*, 'webos25' as webos_version
from kic_data_private.tlamp.rbt_mac_addr_webos25 w25
)

In [0]:
%sql
with tb0 as (
select *
from sandbox.t_weetv.v_po_activation_detail
where 1=1
  and PO_Date>='2023-01-01'
  and Act_Date>='2023-01-01'
  and (PO_Board_Maker != Act_Board_Maker 
    or PO_ODM != Act_ODM or PO_Brand != Act_Brand)
), tb1 as (
  select 
    mac_addr as MAC_ORIGIN
    , mac_addr_hashed as MAC
  from sandbox.z_yeswook_kim.rbt_mac
  union all 
  select 
     mac_addr as MAC_ORIGIN
     , kic_data_ods.tlamp.hash_mac(mac_addr) as MAC
  from kic_data_private.tlamp.activation_date
), tb2 as (
  select 
    distinct MAC_ORIGIN, MAC
  from tb1
)
select *
from (
  select 
    tb2.mac_origin
    , tb0.*
    , DATEDIFF(tb0.Act_Date, tb0.PO_Date) AS day_diff
    , row_number() over (order by PO_date_ym, PO_Date, tb2.mac_origin) as rn
  from tb0 
  left join tb2 USING (MAC)
) t
where rn between 800001 and 1600000
order by PO_date_ym, PO_Date, mac_origin

In [0]:
%sql
with tb0 as (
select *
from sandbox.t_weetv.v_po_activation_detail
where 1=1
  and PO_Date>='2023-01-01'
  -- and Act_Date>='2023-01-01'
  and (PO_Board_Maker != Act_Board_Maker 
    or PO_ODM != Act_ODM)
), tb1 as (
  select 
    mac_addr as MAC_ORIGIN
    , mac_addr_hashed as MAC
  from sandbox.z_yeswook_kim.rbt_mac
  union all 
  select 
     mac_addr as MAC_ORIGIN
     , kic_data_ods.tlamp.hash_mac(mac_addr) as MAC
  from kic_data_private.tlamp.activation_date
), tb2 as (
  select 
    distinct MAC_ORIGIN, MAC
  from tb1
)
select *
from (
  select 
    tb2.mac_origin
    , tb0.*
    , DATEDIFF(tb0.Act_Date, tb0.PO_Date) AS day_diff
    , row_number() over (order by PO_date_ym, PO_Date, tb2.mac_origin) as rn
  from tb0 
  left join tb2 USING (MAC)
) t
where rn between 1 and 8000000
-- where rn between 1600001 and 2400000
order by PO_date_ym, PO_Date, mac_origin

In [0]:
%sql
select 
  date(he_etl_dt), count(1)
from kic_data_ods.tlamp.activation_date
where 1=1
  and len(Platform_code) = len('P000000029')
group by date(he_etl_dt)
order by 1

In [0]:
%sql
select 
  min(he_etl_dt)
from kic_data_ods.tlamp.activation_date
where 1=1
  -- and len(Platform_code) = len('P000000029')
  and Country_code is not null
  -- and Product_CODE is not null
-- group by date(he_etl_dt)
order by 1

In [0]:
%sql
select min(he_etl_dt)
from kic_data_ods.tlamp.activation_date
where 1=1
  and len(Platform_code) = len('P000000029')

In [0]:
%sql
select 
  Platform_code, count(1), count(distinct mac_addr)
from kic_data_ods.tlamp.activation_date
where 1=1
  and len(Platform_code)>4
group by 
  Platform_code
order by 2 desc, 3 desc 